Quadratic Sieve Implementation

In [ ]:
import math
from itertools import product
import numpy as np
import time

Step 1: Select an integer *n* and a corresponding bound B.

In [ ]:
n = 16921456439215439701
#n = 338728169412940137585419303
#n = 46839566299936919234246726809

# Set the bound according to n

def little_o_of_1(n):
    return 1 / math.log(n)

temp = math.log(n)*math.log(math.log(n))
B = int(math.exp(float(0.5 + little_o_of_1(n)) * math.sqrt(temp)))
smoothness_step_bound = int(math.exp(float(1.0 + little_o_of_1(n)) * math.sqrt(temp)))

print(B)

871


Step 2: Get all primes up to B to form set S of primes.


In [ ]:
def sieve_of_atkin(limit):
    if limit < 2:
        return []

    # Initialization of the sieve array
    sieve = [False] * (limit + 1)
    sqrt_limit = int(limit ** 0.5)

    # Step 1: Mark sieve[n] is True where
    # n = (4*x*x)+(y*y) has odd number of solutions, x and y are natural numbers
    # n = (3*x*x)+(y*y) has odd number of solutions, x and y are natural numbers
    # n = (3*x*x)-(y*y) has odd number of solutions, x > y and x and y are natural numbers
    for x in range(1, sqrt_limit + 1):
        for y in range(1, sqrt_limit + 1):
            n = 4*x*x + y*y
            if n <= limit and (n % 12 == 1 or n % 12 == 5):
                sieve[n] = not sieve[n]

            n = 3*x*x + y*y
            if n <= limit and n % 12 == 7:
                sieve[n] = not sieve[n]

            n = 3*x*x - y*y
            if x > y and n <= limit and n % 12 == 11:
                sieve[n] = not sieve[n]

    # Step 2: Mark all multiples of squares of primes
    for a in range(5, sqrt_limit + 1):
        if sieve[a]:
            for b in range(a*a, limit + 1, a*a):
                sieve[b] = False

    # Step 3: Construct the list of primes
    primes = [2]  # Include 2 as it satisfies Euler's criterion
    primes += [i for i in range(3, limit + 1) if sieve[i]]

    return primes
S = sieve_of_atkin(B)
print(S)
print (len(S))


[2, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863]
149


Step 3: Find B-smooth squares.
  - As part of the process, we create the intended matrix of prime powers for each B-smooth value, used to find x^2 equiv y^2.

In [ ]:
def gen_x(n, p, r):
    sqrt_n = math.sqrt((1 + 2 * r) * n)
    sqrt_2n = math.sqrt((2 + 2 * r) * n)
    return [math.ceil(sqrt_n + i) for i in range(p + 1)] + \
           [math.ceil(sqrt_2n + i) for i in range(p + 1)]

def find_b_smooths(n, S, B):
    log_n = math.log(n)
    primes = [p for p in S if p <= B]
    log_primes = {p: math.log(p) for p in primes}
    log_threshold = log_n - 0.5

    smooth_set = set()  # Using a set to store unique smooth numbers
    smooth_set_list = []
    power_matrix = []
    r = 0
    target_smooth_count =  1.3 * len(primes)  # Increase to ensure enough smooth numbers
    print(target_smooth_count)

    while len(smooth_set_list) < target_smooth_count:
        x_list = gen_x(n, 10, r)  # Generate numbers that are likely candidates for being smooth
        for x in x_list:
            xsq = x**2 % n
            log_xsq = math.log(xsq) if xsq > 1 else 0
            powers = [0] * len(primes)
            temp_xsq = xsq

            if log_xsq > log_threshold:
                continue  # Skip values too large to be B-smooth

            for j, prime in enumerate(primes):
                while temp_xsq % prime == 0:
                    powers[j] += 1
                    temp_xsq //= prime
                    log_xsq -= log_primes[prime]
                    if log_xsq < -0.5:  # Stop if the remaining value is too small to match the threshold
                        break
                if log_xsq < -0.5:
                    break

            if temp_xsq == 1 and x not in smooth_set:
                power_matrix.append(powers)
                smooth_set_list.append(x)# Add to set, ensuring uniqueness
                smooth_set.add(x)
                #print(x)  # Optional: for debugging to see the smooth numbers found
        #print(len(smooth_set_list))
        r += 1  # Increment to expand the search in further iterations

    return smooth_set_list, power_matrix
# Track the start time
start_time = time.time()

smooth_list, power_matrix = find_b_smooths(n, S, B)

# Track the end time
end_time = time.time()

# Calculate the runtime
runtime = end_time - start_time
print("Runtime:", runtime, "seconds")

print(len(smooth_list))
#print(power_matrix)


193.70000000000002
Runtime: 2.3583056926727295 seconds
194
[[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [3, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Step 4: Row-reduce matrix and find a linear dependence.

In [ ]:
A = np.array(power_matrix)
power_copy = A.copy()
A = (np.transpose(A)) % 2

def gaussian_elimination(mat):
    num_rows, num_cols = mat.shape
    row = 0
    pivot_positions = []  # Track the positions of pivots
    for col in range(num_cols):
        if row >= num_rows:
            break
        pivot_row = None
        for r in range(row, num_rows):
            if mat[r, col] == 1:
                pivot_row = r
                break
        if pivot_row is not None:
            mat[[row, pivot_row]] = mat[[pivot_row, row]]  # Swap rows
            for r in range(num_rows):
                if r != row and mat[r, col] == 1:
                    mat[r] ^= mat[row]  # Update the matrix using XOR
            pivot_positions.append((row, col))
            row += 1
    return mat[:row], pivot_positions

def find_null_space(mat, pivot_positions):
    num_rows, num_cols = mat.shape
    free_vars = [j for j in range(num_cols) if all(j != col for _, col in pivot_positions)]
    null_space_basis = []
    for free_var in free_vars:
        vec = np.zeros(num_cols, dtype=int)
        vec[free_var] = 1
        for i, col in pivot_positions:
            if i < num_rows:
                vec[col] = mat[i, free_var]
        null_space_basis.append(vec)
    return null_space_basis

result, pivot_positions = gaussian_elimination(A)
#print("Reduced row echelon form:")
#print(result)

null_space_basis = find_null_space(result, pivot_positions)
#print("Null space basis:")
#for vec in null_space_basis:
 #   print(vec)

# Step 5

for v in null_space_basis:
    power_matrix_subset = [x for x, check in zip(power_matrix, v) if check == 1]
    x_list = [x for x, check in zip(smooth_list, v) if check == 1]
    x_final = 1
    for num in x_list:
        x_final = (x_final * num) % n
    '''
    print('x and y mod n...')
    print(x_final)
    '''
    y_final = 1
    for col_index, col in enumerate(zip(*power_matrix_subset)):
        column_sum = sum(col)
        column_product = S[col_index] ** (column_sum // 2)
        y_final = (y_final * column_product) % n

    print(y_final)
    print('x^2 and y^2 mod n...')
    print((x_final ** 2) % n)
    print((y_final ** 2) % n)

    print('factors...')

    factor = math.gcd(x_final - y_final, n)
    print(factor)
    if factor != 1 and factor != n:
        other_factor = n // factor
        print(other_factor)
        break
else:
    print("No non-trivial factors found.")

1131854282
x^2 and y^2 mod n...
1281094115681735524
1281094115681735524
factors...
16921456439215439701
40234766930
x^2 and y^2 mod n...
11298108185954853305
11298108185954853305
factors...
16921456439215439701
520073338928
x^2 and y^2 mod n...
3718139298772008400
3718139298772008400
factors...
16921456439215439701
479910162910
x^2 and y^2 mod n...
12742326580605337490
12742326580605337490
factors...
16921456439215439701
290527061950
x^2 and y^2 mod n...
1749006492524573912
1749006492524573912
factors...
16921456439215439701
2263708564
x^2 and y^2 mod n...
5124376462726942096
5124376462726942096
factors...
16921456439215439701
348704923798
x^2 and y^2 mod n...
14459365206052493119
14459365206052493119
factors...
16921456439215439701
37189713332
x^2 and y^2 mod n...
12436806139887926443
12436806139887926443
factors...
16921456439215439701
2829635705
x^2 and y^2 mod n...
8006838223010847025
8006838223010847025
factors...
16921456439215439701
814584992378
x^2 and y^2 mod n...
763845651128